# Corporate Action Service - Dividends & Events

Track corporate actions including dividend announcements, shareholder meetings (AGM/EGM), and other corporate events.

## Overview

The **Corporate Action Service** tracks important corporate events:

- **Dividend Tracking**: XD dates, payment dates, dividend amounts
- **Shareholder Meetings**: AGM (Annual General Meeting) and EGM (Extraordinary General Meeting)
- **Meeting Details**: Agenda, venue, meeting dates
- **Dividend Capture**: Plan dividend capture strategies
- **Calendar Management**: Track upcoming corporate events

### When to Use This Service

- Dividend income strategies
- Shareholder rights management
- Corporate event calendars
- Ex-dividend date tracking
- Meeting participation planning

In [1]:
!pip install settfex

zsh:1: command not found: pip


In [2]:
import asyncio
from settfex.services.set import get_corporate_actions
import pandas as pd
from datetime import datetime

print("Libraries imported!")

Libraries imported!


## Basic Usage

In [3]:
# Fetch corporate actions
actions = await get_corporate_actions("AOT")

print(f"Found {len(actions)} corporate actions for AOT\n")

for action in actions[:5]:  # Show first 5
    print(f"Type: {action.ca_type}")
    print(f"XD Date: {action.x_date}")
    
    if action.ca_type == "XD":  # Dividend
        print(f"Dividend: {action.dividend} {action.currency}")
        print(f"Payment Date: {action.payment_date}")
        print(f"Source: {action.source_of_dividend}")
    elif action.ca_type == "XM":  # Meeting
        print(f"Meeting: {action.meeting_type}")
        print(f"Date: {action.meeting_date}")
        print(f"Agenda: {action.agenda}")
    
    print("-" * 60)

2025-10-05 20:51:06 | INFO     | settfex.services.set.stock.corporate_action:__init__:108 | CorporateActionService initialized with base_url=https://www.set.or.th
2025-10-05 20:51:06 | INFO     | settfex.services.set.stock.corporate_action:fetch_corporate_actions:151 | Fetching corporate actions for symbol 'AOT' (lang=en) from https://www.set.or.th/api/set/stock/AOT/corporate-action?lang=en
2025-10-05 20:51:06 | INFO     | settfex.utils.data_fetcher:__init__:113 | AsyncDataFetcher initialized with browser=chrome120, timeout=30s, rate_limit=0.0s
2025-10-05 20:51:06 | INFO     | settfex.utils.session_manager:__init__:98 | SessionManager created with browser=chrome120, warmup_site=set, cache=enabled
2025-10-05 20:51:06 | INFO     | settfex.utils.session_cache:__init__:76 | SessionCache initialized at /Users/sarat/.settfex/cache (ttl=3600s, size_limit=100MB)
2025-10-05 20:51:06 | SUCCESS  | settfex.utils.session_manager:_try_load_from_cache:202 | ✓ Loaded session from cache: 4 cookies (age

Found 8 corporate actions for AOT

Type: XD
XD Date: 2024-12-04 00:00:00+07:00
Dividend: 0.79 Baht
Payment Date: 2025-02-06 00:00:00+07:00
Source: Net Profit
------------------------------------------------------------
Type: XM
XD Date: 2024-12-04 00:00:00+07:00
Meeting: AGM
Date: 2025-01-24 14:00:00+07:00
Agenda: Cash dividend payment,Changing The director(s)
------------------------------------------------------------
Type: XD
XD Date: 2023-12-08 00:00:00+07:00
Dividend: 0.36 Baht
Payment Date: 2024-02-14 00:00:00+07:00
Source: Net Profit
------------------------------------------------------------
Type: XM
XD Date: 2023-12-08 00:00:00+07:00
Meeting: AGM
Date: 2024-01-30 14:00:00+07:00
Agenda: Cash dividend payment,Changing the director(s)
------------------------------------------------------------
Type: XM
XD Date: 2022-12-09 00:00:00+07:00
Meeting: AGM
Date: 2023-01-20 14:00:00+07:00
Agenda: Omitted dividend payment,Changing the director(s)
----------------------------------------

## Advanced Usage - Dividend Calendar

In [ ]:
async def build_dividend_calendar(symbols: list[str]):
    """
    Build a dividend calendar for multiple stocks.
    """
    tasks = [get_corporate_actions(symbol) for symbol in symbols]
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    dividends = []
    for symbol, actions in zip(symbols, results):
        if isinstance(actions, Exception):
            continue
        
        for action in actions:
            if action.ca_type == "XD" and action.dividend:
                dividends.append({
                    "symbol": symbol,
                    "xd_date": action.x_date,
                    "payment_date": action.payment_date,
                    "dividend": action.dividend,
                    "currency": action.currency,
                    "source": action.source_of_dividend,
                    "type": action.dividend_type
                })
    
    df = pd.DataFrame(dividends)
    if not df.empty:
        df = df.sort_values('xd_date', ascending=False)
    return df

# Build calendar
symbols = ["PTT", "KBANK", "CPALL", "AOT", "BBL"]
div_calendar = await build_dividend_calendar(symbols)

print(f"Dividend Calendar ({len(div_calendar)} events):\n")
print(div_calendar.head(20).to_string(index=False))

## Use Case: Upcoming Dividends

In [5]:
def find_upcoming_dividends(div_calendar, days_ahead: int = 30):
    """
    Find dividends with XD dates in the next N days.
    """
    today = datetime.now().date()
    
    # Filter upcoming
    df = div_calendar.copy()
    df['xd_date'] = pd.to_datetime(df['xd_date']).dt.date
    df['days_until'] = (df['xd_date'] - today).dt.days
    
    upcoming = df[df['days_until'].between(0, days_ahead)].sort_values('days_until')
    return upcoming

# Find upcoming dividends
upcoming = find_upcoming_dividends(div_calendar, days_ahead=60)
print(f"Upcoming Dividends (next 60 days):\n")
if not upcoming.empty:
    print(upcoming[['symbol', 'xd_date', 'dividend', 'payment_date', 'days_until']].to_string(index=False))
else:
    print("No upcoming dividends found")

AttributeError: Can only use .dt accessor with datetimelike values

## Use Case: Meeting Tracker

In [ ]:
async def track_shareholder_meetings(symbols: list[str]):
    """
    Track upcoming shareholder meetings (AGM/EGM).
    """
    tasks = [get_corporate_actions(symbol) for symbol in symbols]
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    meetings = []
    for symbol, actions in zip(symbols, results):
        if isinstance(actions, Exception):
            continue
        
        for action in actions:
            if action.ca_type == "XM":  # Meeting
                meetings.append({
                    "symbol": symbol,
                    "meeting_type": action.meeting_type,
                    "meeting_date": action.meeting_date,
                    "book_close": action.book_close_date,
                    "agenda": action.agenda[:50] if action.agenda else "N/A"  # Truncate
                })
    
    return pd.DataFrame(meetings).sort_values('meeting_date', ascending=False)

meetings = await track_shareholder_meetings(symbols)
print(f"Shareholder Meetings ({len(meetings)}):\n")
print(meetings.head(10).to_string(index=False))

## Use Case: Dividend Yield Analysis

In [ ]:
# Calculate annual dividend totals
annual_divs = div_calendar.groupby('symbol')['dividend'].sum().reset_index()
annual_divs.columns = ['symbol', 'total_dividend']
annual_divs = annual_divs.sort_values('total_dividend', ascending=False)

print("Total Dividends by Stock (historical):")
print(annual_divs.to_string(index=False))

## Error Handling

In [ ]:
async def safe_get_corporate_actions(symbol: str):
    try:
        actions = await get_corporate_actions(symbol)
        print(f"✓ {symbol}: {len(actions)} actions")
        return actions
    except Exception as e:
        print(f"✗ {symbol}: {e}")
        return []

await safe_get_corporate_actions("PTT")

## Next Steps

- **[Highlight Data](02_highlight_data.ipynb)** - Current dividend yield
- **[Shareholder](06_shareholder.ipynb)** - Ownership structure
- **[Financial](11_financial.ipynb)** - Financial statements

**Documentation**: `/docs/settfex/services/set/corporate_action.md`